In [32]:
from sklearn.ensemble import GradientBoostingRegressor
# 导入均方误差计算
from sklearn.metrics import mean_squared_error
# 导入标准化模块包
from sklearn.preprocessing import RobustScaler
# 导入划分数据集包，交叉验证包
from sklearn.model_selection import train_test_split,KFold,cross_val_score
# 导入Xgboost算法包
import xgboost as xgb
from  sklearn import datasets 
import pandas as pd 
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing  
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

from sklearn.compose import ColumnTransformer  
from sklearn.preprocessing import OneHotEncoder, StandardScaler  
from sklearn.ensemble import RandomForestClassifier

In [18]:
dataset_df = pd.read_csv('./titanic/train.csv')
test_df = pd.read_csv('./titanic/test.csv')

y = dataset_df.pop('Survived')

In [19]:
features = ["PassengerId","Pclass", "Sex", "SibSp", "Parch"]
train = dataset_df[features]
test = test_df[features]

all_data = pd.concat([train, test],axis=0)


In [20]:
def handle_missing_values(df):
    num_cols = [cname for cname in df.columns if df[cname].dtype in ['int64', 'float64']]
    cat_cols = [cname for cname in df.columns if df[cname].dtype == "object"]
    values = {}
    for a in cat_cols:
        values[a] = 'UNKNOWN'
    for a in num_cols:
        values[a] = df[a].median()
    df.fillna(value=values, inplace=True)
    

handle_missing_values(all_data)

all_data = pd.get_dummies(all_data)

In [21]:
train = all_data[all_data['PassengerId'] < 892]
train.pop('PassengerId')

test = all_data[all_data['PassengerId'] >= 892]
Id = test.pop('PassengerId')

In [36]:
randomForest = RandomForestClassifier(n_estimators = 30,  
                                max_depth=13, class_weight='balanced', n_jobs=-1, 
                                   random_state=0)
randomForest.fit(train, y)

RandomForestClassifier(class_weight='balanced', max_depth=13, n_estimators=30,
                       n_jobs=-1, random_state=0)

In [35]:
cv_params = {'learning_rate':[0.31, 0.311, 0.312]}
other_params = {'learning_rate': 0.31, 'n_estimators': 600, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.6, 'colsample_bytree': 0.6, 'gamma': 0.1, 'reg_alpha': 3, 'reg_lambda': 1, 'objective': 'multi:softmax', 'num_class':2}

model = xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='roc_auc', cv=5, verbose=1, n_jobs=4)

optimized_GBM.fit(train, y)
evalute_result = optimized_GBM.cv_results_['mean_test_score']
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))


print(optimized_GBM.best_estimator_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
每轮迭代运行结果:[0.85081784 0.84910646 0.85050047]
参数的最佳取值：{'learning_rate': 0.31}
最佳模型得分:0.8508178366171866
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.31, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=600, n_jobs=None, num_class=2,
              num_parallel_tree=None, objective='multi:softmax', ...)


In [37]:
pred = randomForest.predict(test)

out = {
    'PassengerId':Id,
    'Survived':pred
}

out = pd.DataFrame(out)

print(out)

out.to_csv('submisson_titanic.csv',index=False)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
